In [1]:
import geopandas as gpd
import pandas as pd
import os
#from shapely.geometry import LineString
from shapely import wkt
import numpy as np
from utils.load_csv import load_csv
from utils.load_shp import load_shp
from utils.save_shp import save_shp
#from TESTE 1:
def manual_data_path(file_name)->str:
    path = os.path.join(
        'data',
        'manual_data',
        file_name
    )
    return path

In [2]:
onibus_diesel_dia31= pd.read_csv(manual_data_path('linestring_2025-12-31_silver.csv'))

In [3]:
BUFFER = 10

In [4]:
df_onibus=onibus_diesel_dia31
df_onibus['geometry'] = df_onibus['geometry'].apply(wkt.loads) #transforma str em geometry
df_onibus.columns

Index(['codigo_onibus', 'geometry'], dtype='str')

Para transformar o df em gdf, precisamos setar a geometria... mas pra isso a coluna precisa ser do tipo geometria. Nossa coluna 'geometry' está como `str` atualmente, então o certo não é colocar como `set_geometry`, mas usar o `wkt` (well known text) do `shapely`

In [5]:
gdf_onibus =gpd.GeoDataFrame(
    df_onibus, 
    geometry= 'geometry',
    crs="EPSG:31983"
)

In [6]:
#escolher ônibus aleatórios

In [7]:
#transformar pontos x e y em geometry

In [8]:
#TRATAR DADOS
#gdf_onibus = gdf_onibus.sort_values(['codigo_onibus', 'hora_gps']).reset_index(drop = True)
#counts = gdf_onibus.codigo_onibus.value_counts()
#remover = counts[counts < 2].index
#gdf_onibus = gdf_onibus[~gdf_onibus.codigo_onibus.isin(remover)]

In [9]:
#LINHAS? linestring-> gdf_trajetos

In [10]:
gdf_buffer = gdf_onibus.sample(15).copy()
gdf_buffer

,codigo_onibus,geometry
6980,75252,"LINESTRING (320312.918 7382879.509, 320312.918..."
3714,47106,"LINESTRING (356203.438 7390761.564, 357326.568..."
7215,77923,"LINESTRING (325753.547 7385718.358, 325613.854..."
5447,63328,"LINESTRING (332243.897 7378694.854, 332409.369..."
7813,81502,"LINESTRING (322552.769 7390602.511, 322859.615..."
321,11834,"LINESTRING (325614.528 7400485.827, 325739.085..."
535,16074,"LINESTRING (322003.482 7402613.316, 322059.854..."
125,11040,"LINESTRING (326218.325 7397956.625, 326218.325..."
6966,73978,"LINESTRING (324730.302 7381647.026, 324202.965..."
2248,31785,"LINESTRING (349324.395 7398326.091, 349324.395..."


In [11]:
gdf_buffer['geometry'] = gdf_buffer.geometry.buffer(BUFFER)
gdf_buffer

,codigo_onibus,geometry
6980,75252,"POLYGON ((320301.068 7382855.752, 320301.097 7..."
3714,47106,"POLYGON ((350374.114 7395481.897, 350501.33 73..."
7215,77923,"POLYGON ((325623.111 7385372.766, 325622.738 7..."
5447,63328,"POLYGON ((333469.597 7379919.635, 333469.6 737..."
7813,81502,"POLYGON ((321192.225 7392747.054, 321192.225 7..."
321,11834,"POLYGON ((324563.784 7406402.222, 324540.963 7..."
535,16074,"POLYGON ((323627.96 7401415.059, 323627.605 74..."
125,11040,"POLYGON ((324474.482 7397704.821, 324473.505 7..."
6966,73978,"POLYGON ((318671.472 7378626.889, 318648.574 7..."
2248,31785,"POLYGON ((349334.395 7398326.091, 349334.347 7..."


In [12]:
#dissolve por codigo_onibus

# 04_overlay_setores_censitários

In [13]:
setores_censitarios_sp = load_shp('setores_censitarios_sp.shp')
setores_censitarios_sp = setores_censitarios_sp.to_crs(gdf_buffer.crs)

In [14]:
gdf_overlay = gpd.overlay(setores_censitarios_sp,
                          gdf_buffer,
                          how = 'intersection')
gdf_overlay = gdf_overlay.explode(index_parts = False).reset_index(drop = True)
len(gdf_overlay)

6772

In [15]:
#tirar geometrias pequenininhas
debuff_value= -(BUFFER**(1/2))
 
gdf_overlay['debuffed'] = gdf_overlay.buffer(debuff_value)

gdf_overlay['debuffed_area']= gdf_overlay['debuffed'].area

gdf_overlay = gdf_overlay[gdf_overlay['debuffed_area']>0]

gdf_overlay = gdf_overlay.drop(columns = {'debuffed', 'debuffed_area'})
len(gdf_overlay)

6202

# 05_calculo_pop_afetada

In [16]:
setores_censitarios_sp = setores_censitarios_sp.to_crs(gdf_overlay.crs)

setores_censitarios_sp['area_setor'] = setores_censitarios_sp.geometry.area

setores_area = setores_censitarios_sp[['CD_SETOR', 'area_setor', 'pop']]

In [17]:
gdf_overlay['area_intersecao'] = gdf_overlay.geometry.area

In [18]:
gdf_overlay = gdf_overlay.merge(setores_area,
                                on = 'CD_SETOR',
                                how = 'left')

In [19]:
gdf_overlay = gdf_overlay.drop(columns = ['pop_y'])

gdf_overlay = gdf_overlay.rename(columns = {'pop_x': 'pop'})

In [20]:
gdf_overlay['pop_afetada'] = round((gdf_overlay['pop'] * (gdf_overlay['area_intersecao'] / gdf_overlay['area_setor']))).astype(int)

In [21]:
gdf_overlay = gdf_overlay.rename(columns = {'area_intersecao': 'area_inter', 
                                            'pop_afetada': 'pop_buffer'})

In [22]:
gdf_overlay.columns

Index(['CD_SETOR', 'AREA_KM2', 'CD_REGIAO', 'NM_REGIAO', 'CD_UF', 'NM_UF',
       'CD_MUN', 'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_SUBDIST', 'NM_SUBDIST',
       'CD_MICRO', 'NM_MICRO', 'CD_MESO', 'NM_MESO', 'CD_RGI', 'NM_RGI',
       'CD_RGINT', 'NM_RGINT', 'CD_CONCURB', 'NM_CONCURB', 'pop', 'v0002',
       'v0003', 'v0004', 'v0005', 'v0006', 'v0007', 'codigo_onibus',
       'geometry', 'area_inter', 'area_setor', 'pop_buffer'],
      dtype='str')

## Testes

In [23]:
setores_uniques = (
    gdf_overlay[['CD_SETOR', 'pop']]
    .drop_duplicates(subset='CD_SETOR')
    .rename(columns={'CD_SETOR': 'name'})
    .reset_index(drop=True)
)

In [24]:
#NÃO QUERO PENSAR NISSO AGORA 
for i, s in setores_uniques.iterrows():
    cd = s['name']
    setor_x = gdf_overlay.loc[gdf_overlay['CD_SETOR'] == cd]
    #soma pop afetada
    setores_uniques.loc[setores_uniques['name'] == cd, 'soma_pop_afetada'] = sum(
        setor_x['pop_buffer']
    )

setores_uniques['subtrac_pop'] = setores_uniques['pop'] - setores_uniques['soma_pop_afetada']

In [25]:
setores_uniques.loc[setores_uniques]

ValueError: Cannot index with multidimensional key

In [ ]:
gdf_buffer['pop_afetada'] = [
    gdf_overlay.loc[gdf_overlay['codigo_onibus'] == cb, 'pop_buffer'].sum()
    for cb in gdf_buffer['codigo_onibus']
]
gdf_buffer['pop_afetada'] = gdf_buffer['codigo_onibus'].map(
    gdf_overlay.groupby('codigo_onibus')['pop_buffer'].sum()
)
gdf_buffer

# Média do dia

In [ ]:
media_pop_dia = gdf_buffer['pop_afetada'].mean()
round(media_pop_dia, 0)